In [20]:
# bank of of potential personal information asked on applications

personal_info_keywords = [ 'First Name', 'Last Name', 'Middle Name', 'Title', 'SSN', 'Personal Cellphone', 'Work Phone', 'Home Phone', 'Adress', 'City',
                 'State', 'Zip', 'Drivers License ID', 'Drivers License Type', 'Height', 'Weight', 'Gender', 'Sex', 'Eye Color', 'Date of Birth',
                 'Race', 'Ethnicity', 'Legal Status', 'Visa Number', 'Passport Number', 'Veteran Status', 'Emergency Person First Name',
                 'Emergency Person Last Name', 'Emergency Person Phone', 'Personal Email', 'Highest Education Level', 'College', 'Graduation date', 
                 'Major', 'Minor', 'GPA', 'Age', 'Sexual Orientation']

In [21]:
len(personal_info_keywords)

38

In [22]:
personal_data_bank = {}
communal_data_bank = {}

#creates a personal database inside of a communal database 
def personal_database_creator(unique_username):
    
    for i in personal_info_keywords:
        data = input('Enter your: {}'.format(i))
        personal_data_bank[i] = data
        
    communal_data_bank[str(unique_username)] = personal_data_bank
    

In [23]:
#ensures that the selected username does not overlap with existing usernames in the communal database
def username_uniqeness_check(username):
    for key in communal_data_bank:
        if key == str(username):
            print('Username taken.')
            return False
    print('Username accepted.')
    return True

In [24]:
import Levenshtein #to compare similar words (personal info keywords to field names)
import selenium #to initialize the browser and read the online application form

In [25]:
#scans the online application and puts the field names into a list
def field_names_gatherer(application_link):

    # Initialize the WebDriver (e.g., Chrome)
    driver = webdriver.Chrome()
    
    # Navigate to the web page with the form
    driver.get(str(application_link))
    
    # Find all input elements within the form
    input_elements = driver.find_elements_by_css_selector("form input")
    
    # Collect the names (or other attributes) into a list
    global field_names
    field_names = [element.get_attribute("name") for element in input_elements]
    
    # Print the collected field names
    print("Form Field Names:", field_names)
    
    # Close the browser when done
    driver.quit()
    return field_names


In [26]:
#create a dictionary that matches the personal info keywords with field names of the online application
def matchings():
    assert len(field_names)==len(personal_info_keywords), 'the list of personal info keywords must have the same length as the list of field names (1-1 correspondence is required)'
    
    fields = field_names
    global field_mapping
    field_mapping = {}
    for keyword in personal_info_keywords:
        dist = 1000
        closest_field = None
        for field in fields:
            distance = Levenshtein.distance(keyword.lower(), field.lower())
            if distance < dist:
                dist = distance
                closest_field = field
                
        field_mapping[keyword] = closest_field
        fields.remove(closest_field)

    print(field_mapping)
    return field_mapping
    

In [27]:
#can also create the personal info keyword list based on the field names of the online application
#first, scan the field names of the online application
#second, create a dictionary with the fields as keys and the users personal info as values
#third, input the values (personal info) into the corresponding fields in the online application

#useful in the scenario when one has to fill out multiple online applications that ask the same things (overlapping fields)
#avoids repetitive typing, saves time

application_based_user_info = {}

def fields_based_info(application_link):

    fields = field_names_gatherer(str(application_link))
    for field in fields:
        entry = input(f'Enter your: {field}.')
        application_based_user_info[field] = entry

    return application_based_user_info



def autofill(application_link):

    driver = webdriver.Chrome()

    driver.get(str(application_link))

    for name in field_names_gatherer(str(application_link)):
        input_field = driver.find_element_by_name(name)
        input_field.send_keys(application_based_user_info[name])
    


    form = driver.find_element_by_id('application-form')
    form.submit()

    driver.quit()
